In [2]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Microsoft YaHei'  

from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from glob import glob
import pandas as pd
import dask
import dask.dataframe as dd
from dask import delayed
import re
import pandas as pd
import jieba

/home/yjia2/thesis/thesis/thesis/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
cluster = SLURMCluster(queue='caslake', cores=20, memory='80GB', 
                       processes=20, walltime='03:00:00', interface='ib0',
                       job_extra_directives=['--account=macs40123']
                      )
cluster.scale(jobs=1)

In [6]:
! squeue -u yjia2

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          38484556   caslake dask-wor    yjia2  R       0:05      1 midway3-0072


In [7]:
client = Client(cluster)
client

<Client: 'tcp://172.25.0.66:35923' processes=20 threads=20, memory=74.60 GiB>

# Data  Clean

In [8]:
xlsx_files = sorted(glob("data/*.xlsx"))
assert xlsx_files, "No .xlsx files found under data/"

In [9]:
def load_excel_fixed(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=0, engine="openpyxl", dtype=str)
    expected_cols = ["年份", "日期", "报纸版次", "标题", "文本内容"]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise ValueError(f"{path} : {missing}")
    df = df[expected_cols]
    df["年份"] = pd.to_numeric(df["年份"], errors="raise").astype("Int64")
    for c in ["日期","报纸版次","标题","文本内容"]:
        df[c] = df[c].astype(str)
    df["年份"] = pd.to_numeric(df["年份"], errors="coerce")
    df = df.dropna(subset=expected_cols)
    return df

In [11]:
delayed_parts = [delayed(load_excel_fixed)(p) for p in xlsx_files]

meta = pd.DataFrame({
    "年份": pd.Series(dtype="Int64"),
    "日期": pd.Series(dtype="object"),
    "报纸版次": pd.Series(dtype="object"),
    "标题": pd.Series(dtype="object"),
    "文本内容": pd.Series(dtype="object"),
})

ddf = dd.from_delayed(delayed_parts, meta=meta)

out_dir = "merged_parquet"
ddf.to_parquet(
    out_dir,
    engine="pyarrow",
    write_index=False,
    partition_on=["年份"]
)

print("✅ Merged and partitioned Parquet written to:", out_dir)

✅ Merged and partitioned Parquet written to: merged_parquet


In [12]:
ddf_check = dd.read_parquet("merged_parquet")
print(ddf_check.npartitions)   # 分区数
print(ddf_check.head())


78
           日期 报纸版次           标题  \
0  1947-01-01  第4版     我军炮兵大显神威   
1  1947-01-01  第2版      中央局边府军区   
2  1947-01-01  第4版   山东军区一年自卫作战   
3  1947-01-01  第4版      不投降就消灭它   
4  1947-01-01  第3版  新年试谈副刊和群众结合   

                                                文本内容    年份  
0   我军炮兵大显神威 配合步兵迅速解决战斗 刘竞 1947-01-01 第4版() 专栏： 我...  1947  
1   中央局边府军区 新年慰贺民主建国军西北民主联军全体将士 1947-01-01 第2版() ...  1947  
2   山东军区一年自卫作战 共歼蒋军十二万余 今日为山东解放军建军九周年纪念日 1947-01-...  1947  
3   不投降就消灭它！ 蒋军九十六师师部被歼经过 萧里 1947-01-01 第4版() 专栏：...  1947  
4   新年试谈副刊和群众结合 林曦 1947-01-01 第3版() 专栏： 新年试谈副刊和群众...  1947  


In [15]:
# —— 需要删除的“马克数据网”相关字段
MARK_PATTERNS = [
    r"数据由“马 克 团 队”整理",
    r"来源：百度搜索“马克数据网”",
    r"（更多数据，详见马克数据网）",
    r"详见官网",
    r"来自马克数据网",
    r"该数据由【马克数据网】整理",
    r"关注公众号“马克数据网”",
    r"更多数据：搜索“马克数据网”",
    r"（来自 马克数据网）",
    r"马克数据网"
]

# —— 常见标点（中文 + 英文）
RE_PUNCT = re.compile(r"[，。！？、；：“”‘’（）《》〈〉【】『』「」—…·,.!?;:\"'()\-]+")
RE_BRACKET = re.compile(r"【[^】]*】")

with open("baidu_stopwords.txt", "r", encoding="utf-8") as f:
    STOPWORDS = set([line.strip() for line in f if line.strip()])

def clean_and_segment(text: str, year_val) -> str:
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)

    # 1️⃣ 删除【】及其中内容
    text = RE_BRACKET.sub("", text)

    # 2️⃣ 年份 ≤ 2002 删除专栏标题前缀
    try:
        y = int(year_val)
    except Exception:
        y = None
    if y is not None and y <= 2002:
        text = re.sub(r"^.*?专栏[:：]", "", text)

    # 3️⃣ 删除“马克数据网”相关字段
    for pat in MARK_PATTERNS:
        text = re.sub(pat, "", text)

    # 4️⃣ 用标点作为分割符来“辅助分词”
    text = RE_PUNCT.sub(" ", text)   # 先将标点替换成空格

    # 5️⃣ jieba 分词
    seg_list = jieba.cut(text)
    seg_text = " ".join(seg_list)

    # 6️⃣ 删除所有非汉字字符（包括标点、数字、英文字母）
    seg_text = re.sub(r"[^ \u4e00-\u9fff]", "", seg_text)

    # 7️⃣ 再次规整空格
    seg_text = re.sub(r"\s+", " ", seg_text).strip()

    tokens = [w for w in seg_text.split() if w not in STOPWORDS]
    seg_text = " ".join(tokens)

    return seg_text


In [17]:
import dask.dataframe as dd

ddf = dd.read_parquet("merged_parquet")

# 在每个分区上并行执行分词清洗
ddf = ddf.assign(
    文本内容_清洗 = ddf.map_partitions(
        lambda df: df.apply(
            lambda row: clean_and_segment(row["文本内容"], row["年份"]),
            axis=1
        ),
        meta=("文本内容_清洗", "object")
    )
)

# 写出新的 Parquet（保留年份分区）
out_dir = "cleaned_segmented_parquet"
ddf.to_parquet(
    out_dir,
    engine="pyarrow",
    write_index=False,
    partition_on=["年份"]
)
print("✅ Cleaned and segmented text written to:", out_dir)


✅ Cleaned and segmented text written to: cleaned_segmented_parquet


In [18]:
import dask.dataframe as dd

# 重新读取刚写出的 parquet 文件
ddf_cleaned = dd.read_parquet("cleaned_segmented_parquet")

# 查看前几行（会触发少量计算）
print(ddf_cleaned.head(5))


           日期 报纸版次           标题  \
0  1947-01-01  第4版     我军炮兵大显神威   
1  1947-01-01  第2版      中央局边府军区   
2  1947-01-01  第4版   山东军区一年自卫作战   
3  1947-01-01  第4版      不投降就消灭它   
4  1947-01-01  第3版  新年试谈副刊和群众结合   

                                                文本内容  \
0   我军炮兵大显神威 配合步兵迅速解决战斗 刘竞 1947-01-01 第4版() 专栏： 我...   
1   中央局边府军区 新年慰贺民主建国军西北民主联军全体将士 1947-01-01 第2版() ...   
2   山东军区一年自卫作战 共歼蒋军十二万余 今日为山东解放军建军九周年纪念日 1947-01-...   
3   不投降就消灭它！ 蒋军九十六师师部被歼经过 萧里 1947-01-01 第4版() 专栏：...   
4   新年试谈副刊和群众结合 林曦 1947-01-01 第3版() 专栏： 新年试谈副刊和群众...   

                                             文本内容_清洗    年份  
0  我军 炮兵 大显神威 配合 步兵 解决 刘竞 此次 苏北 大 歼灭战 中 新四军 炮兵 大显...  1947  
1  中央局 边府 军区 新年 慰贺 民主 建国 军 西北 民主 联军 全体 将士 中央局 边区 ...  1947  
2  山东 军区 一年 自卫 作战 共歼 蒋军 十二 万余 今日 山东 解放军 建军 九 周年 纪...  1947  
3  不 投降 消灭 蒋军 九十六 师师 部 被歼 萧里 前线 记者 萧里 报导 人 圩 宿迁 北...  1947  
4  新年 试谈 副刊 群众 林曦 元旦 照例 一篇 新年 献辞 空话 无益 谈谈 都 关系 一个...  1947  
